In [257]:
import pickle
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
import numpy as np 
import pandas as pd
import re
from word2number import w2n
from sklearn.metrics import mean_absolute_error

In [237]:
#Read the preprocessed data

In [238]:
# with open('./data_for_tree.pkl', 'rb') as f:
#     data = pickle.load(f)

In [239]:
data = pd.read_csv('./Data/CarPrice_Assignment.csv')

In [240]:
data.describe()

,car_ID,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,103.000000,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,10.142537,104.117073,5125.121951,25.219512,30.751220,13276.710571
std,59.322565,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,3.972040,39.544167,476.985643,6.542142,6.886443,7988.852332
min,1.000000,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,52.000000,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7788.000000
50%,103.000000,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,154.000000,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.580000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16503.000000
max,205.000000,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


In [262]:
data.skew().sort_values()

/tmp/ipykernel_24681/3072646498.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data.skew().sort_values()


enginelocation_front    -8.143531
fueltype_gas            -2.732619
enginetype_ohc          -0.998089
stroke                  -0.689705
drivewheel_fwd          -0.349111
doornumber              -0.247552
boreratio                0.020156
carheight                0.063123
peakrpm                  0.075159
carbody_sedan            0.128024
carlength                0.155954
fuelsystem_mpfi          0.167656
symboling                0.211072
drivewheel_rwd           0.539225
highwaympg               0.539997
citympg                  0.663704
carbody_hatchback        0.673587
curbweight               0.681398
fuelsystem_2bbl          0.767785
carwidth                 0.904003
wheelbase                1.050214
horsepower               1.405310
price                    1.777678
enginesize               1.947655
carbody_wagon            2.327670
compressionratio         2.610862
fuelsystem_idi           2.732619
fueltype_diesel          2.732619
cylindernumber           2.817459
enginetype_ohc

In [241]:
data = pd.get_dummies(data, columns = ['carbody', 'drivewheel', 'enginetype','fuelsystem', 'enginelocation', 'fueltype'])

In [242]:
def num_encoder(x):
    return w2n.word_to_num(x)


data['doornumber'] = data['doornumber'].map(lambda x: w2n.word_to_num(x))
data['cylindernumber'] = data['cylindernumber'].map(lambda x:w2n.word_to_num(x))

In [244]:
numeric_features = ['price','wheelbase', 'carlength', 'carwidth', 'carheight', 'curbweight', 'enginesize', 'boreratio', 'stroke','compressionratio','horsepower','peakrpm','citympg','highwaympg']
scaler = MinMaxScaler()
for col in numeric_features:
    data[col] = scaler.fit_transform(data[[col]])

In [263]:
data.skew().sort_values()

/tmp/ipykernel_24681/3072646498.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data.skew().sort_values()


enginelocation_front    -8.143531
fueltype_gas            -2.732619
enginetype_ohc          -0.998089
stroke                  -0.689705
drivewheel_fwd          -0.349111
doornumber              -0.247552
boreratio                0.020156
carheight                0.063123
peakrpm                  0.075159
carbody_sedan            0.128024
carlength                0.155954
fuelsystem_mpfi          0.167656
symboling                0.211072
drivewheel_rwd           0.539225
highwaympg               0.539997
citympg                  0.663704
carbody_hatchback        0.673587
curbweight               0.681398
fuelsystem_2bbl          0.767785
carwidth                 0.904003
wheelbase                1.050214
horsepower               1.405310
price                    1.777678
enginesize               1.947655
carbody_wagon            2.327670
compressionratio         2.610862
fuelsystem_idi           2.732619
fueltype_diesel          2.732619
cylindernumber           2.817459
enginetype_ohc

In [245]:
data['CarName_first'] = data['CarName'].map(lambda x: re.split(r'\s|-', x)[0])

In [246]:
data = data.loc[:, ~data.columns.isin(['car_ID', 'CarName', 'aspiration'])]

In [247]:
test_data = data.groupby('CarName_first').head(6)


In [248]:
indexes = ~data.index.isin(test_data.index)
train_data = data.loc[indexes]
train_data.columns

Index(['symboling', 'doornumber', 'wheelbase', 'carlength', 'carwidth',
       'carheight', 'curbweight', 'cylindernumber', 'enginesize', 'boreratio',
       'stroke', 'compressionratio', 'horsepower', 'peakrpm', 'citympg',
       'highwaympg', 'price', 'carbody_convertible', 'carbody_hardtop',
       'carbody_hatchback', 'carbody_sedan', 'carbody_wagon', 'drivewheel_4wd',
       'drivewheel_fwd', 'drivewheel_rwd', 'enginetype_dohc',
       'enginetype_dohcv', 'enginetype_l', 'enginetype_ohc', 'enginetype_ohcf',
       'enginetype_ohcv', 'enginetype_rotor', 'fuelsystem_1bbl',
       'fuelsystem_2bbl', 'fuelsystem_4bbl', 'fuelsystem_idi',
       'fuelsystem_mfi', 'fuelsystem_mpfi', 'fuelsystem_spdi',
       'fuelsystem_spfi', 'enginelocation_front', 'enginelocation_rear',
       'fueltype_diesel', 'fueltype_gas', 'CarName_first'],
      dtype='object')

In [249]:
train_y = train_data['price']
test_y = test_data['price']

In [250]:
train_X = train_data.loc[:, ~train_data.columns.isin(['CarName_first', 'price' ])]

In [251]:
test_X = test_data.loc[:, ~test_data.columns.isin(['CarName_first', 'price'])]

In [252]:
train_X

,symboling,doornumber,wheelbase,carlength,carwidth,carheight,curbweight,cylindernumber,enginesize,boreratio,...,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi,enginelocation_front,enginelocation_rear,fueltype_diesel,fueltype_gas
9,0,2,0.376093,0.553731,0.633333,0.350000,0.607060,5,0.264151,0.421429,...,0,0,0,1,0,0,1,0,0,1
16,0,2,0.492711,0.786567,0.633333,0.491667,0.733902,6,0.558491,0.771429,...,0,0,0,1,0,0,1,0,0,1
17,0,4,0.682216,0.834328,0.883333,0.708333,0.782389,6,0.558491,0.771429,...,0,0,0,1,0,0,1,0,0,1
27,1,2,0.206997,0.241791,0.291667,0.233333,0.272692,4,0.139623,0.350000,...,0,0,0,1,0,0,1,0,0,1
28,-1,4,0.486880,0.500000,0.358333,1.000000,0.406129,4,0.230189,0.571429,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,4,0.655977,0.711940,0.716667,0.641667,0.567882,4,0.301887,0.885714,...,0,0,0,1,0,0,1,0,0,1
201,-1,4,0.655977,0.711940,0.708333,0.641667,0.605508,4,0.301887,0.885714,...,0,0,0,1,0,0,1,0,0,1
202,-1,4,0.655977,0.711940,0.716667,0.641667,0.591156,6,0.422642,0.742857,...,0,0,0,1,0,0,1,0,0,1
203,-1,4,0.655977,0.711940,0.716667,0.641667,0.670675,6,0.316981,0.335714,...,0,1,0,0,0,0,1,0,1,0


In [253]:
reg = DecisionTreeRegressor(random_state=0).fit(train_X, train_y)

In [258]:
y_pred_train = reg.predict(train_X)
mae_train = mean_absolute_error(train_y, y_pred_train)
mae_train

0.043425428902791434

In [259]:
y_pred_test = reg.predict(test_X)
mae_test = mean_absolute_error(test_y, y_pred_test)
mae_test

0.10984930213002551

In [255]:
reg2 = MLPRegressor().fit(train_X, train_y)

In [260]:
y_pred_train = reg2.predict(train_X)
mae_train = mean_absolute_error(train_y, y_pred_train)
mae_train

0.38107177624171185

In [261]:
y_pred_test = reg2.predict(test_X)
mae_test = mean_absolute_error(test_y, y_pred_test)
mae_test

0.35951126309764736